## Imports

In [1]:
import os
import cv2
import pickle
import imageio
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.use('Agg')
from matplotlib.animation import FFMpegWriter
from tqdm import tqdm
from pprint import pprint
import sys
sys.path.append('./')
from utils import normalize_image, get_sequences

In [2]:
from enum import Enum
from collections import namedtuple

class ModelType(Enum):
    CLASSIFICATION = 0
    SEGMENTATION = 1

classlabels_viz_colors = ['black', 'green', 'yellow', 'blue', 'red', 'magenta', 'cyan',
                          'lightseagreen', 'brown', 'magenta', 'olive', 'wheat', 'white', 'black']
classlabels_viz_bounds = [-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 100]

classlabels_viz_cmap = mpl.colors.ListedColormap(classlabels_viz_colors)
classlabels_viz_norm = mpl.colors.BoundaryNorm(classlabels_viz_bounds, classlabels_viz_cmap.N)

confidence_heatmap_viz_colors = ['black', 'blue', 'red', 'orange', 'yellow', 'lightgreen', 'lightseagreen']
confidence_heatmap_viz_bounds = [-1, 0,0.5,0.6,0.7,0.8,0.9,1]
confidence_heatmap_viz_cmap = mpl.colors.ListedColormap(confidence_heatmap_viz_colors)
confidence_heatmap_viz_norm = mpl.colors.BoundaryNorm(confidence_heatmap_viz_bounds, confidence_heatmap_viz_cmap.N)


LabelColor = namedtuple('LabelColor', ['name', 'id', 'trainid', 'color', 'category'])

LABEL_COLORS = [
    LabelColor('class1', 1, 0, (128, 0, 128), 'driveableterrain'),
    LabelColor('class2', 2, 1, (255, 0, 0), 'non-driveableterrain'),
    LabelColor('class3', 3, 2, (0, 0, 255), 'sky'),
    LabelColor('class4', 4, 3, (0, 255, 0), 'trees'),
    LabelColor('class5', 5, 4, (255, 0, 255), 'implement'),
    LabelColor('class6', 6, 5, (255, 255, 0), 'basket markers')
]

LABEL_COLORS_4CLASS = LABEL_COLORS[0:4]
LABEL_COLORS_5CLASS = LABEL_COLORS[0:5]
LABEL_COLORS_6CLASS = LABEL_COLORS[0:6]
LABEL_COLORS_SKY_DET = [LABEL_COLORS[0], LABEL_COLORS[2]]

LABEL_COLORS_IMPL = [
    LabelColor('class1', 1, 1, (128, 0, 128), 'implement'),
    LabelColor('class2', 2, 2, (255, 0, 0), 'sweep'),
    LabelColor('class3', 3, 3, (0, 0, 255), 'harrow_tine'),
    LabelColor('class4', 4, 4, (0, 255, 0), 'basket'),
    LabelColor('class5', 5, 5, (0, 255, 0), 'basket_marker'),
    LabelColor('class6', 0, 255, (0, 0, 0), 'ignore')
]

LABEL_COLORS_IMPL_REDUCED = [
    LabelColor('class0', 1, 0, (0, 0, 0), 'background'),
    LabelColor('class1', 2, 1, (0, 255, 0), 'implement'),
    LabelColor('class2', 3, 2, (255, 0, 0), 'sweep'),
    LabelColor('class3', 4, 3, (0, 255, 0), 'basket_marker'),
    LabelColor('class6', 0, 255, (0, 0, 0), 'ignore')
]

PLUG_LABEL_MAP ={0: 'no-plug', 1: 'plug'}
IMPL_SEGMENT_LABEL_MAP = {0: 'background', 1: 'implement', 2: 'sweep', 3:'basket_marker'}

In [18]:
def create_mpl_viz_outputs(output_path,
                           image,
                           prediction_labels,
                           confidences,
                           depth_img, 
                           image_title='Image', 
                           pred_title='Prediction', 
                           conf_title='Confidence', 
                           depth_title='Depth',
                           bbox_coords=[]):
    """
    Utility function to plot results based on order of input provided.

    Axes index can have different values based on input provided.

    For example: If image, prediction and groundtruth_label is provided, A three axes plot will be generated with
    image(ax1), ground_truth(ax2), prediction(ax3).

    Order of plot if all the inputs are provided will be in same order as arguments listed above.
    """
    axis_index = list(range(len(list(filter(lambda x: x is not None, [image,
                                                                      prediction_labels, confidences,
                                                                      depth_img])))))
    axis_curr_index = 0
    fig, axes = mpl.pyplot.subplots(1, len(axis_index), figsize=((60, 30)))
    if bbox_coords:
#         xmin, ymin, xmax, ymax = bbox_coords
        xmin, xmax, ymin, ymax = bbox_coords
        if (xmin < 0) or (ymin < 0):
            raise ValueError(f'Either {xmin} or {ymin} are negative')
        else:
            rect = mpl.patches.Rectangle((ymin, xmin), (ymax - ymin), (xmax - xmin), linewidth=3, edgecolor='k',
                                     facecolor='none')
    else:
        rect = None

    if axis_curr_index < len(axes):
        # Grab only RGB channels from image, otherwise depth with distort the image when it is displayed
        axes[axis_curr_index].imshow(image)
        axes[axis_curr_index].set_title(image_title, fontsize=30)
        axes[axis_curr_index].axis('off')
        axis_curr_index += 1

    if axis_curr_index < len(axes):
        axes[axis_curr_index].imshow(depth_img, cmap='turbo')
        axes[axis_curr_index].set_title(depth_title, fontsize=30)
        axes[axis_curr_index].axis('off')
        axis_curr_index += 1

    if axis_curr_index < len(axes):
        axes[axis_curr_index].imshow(prediction_labels, classlabels_viz_cmap, classlabels_viz_norm, interpolation='nearest')
        if rect is not None:
            rect1 = mpl.patches.Rectangle((ymin, xmin), (ymax - ymin), (xmax - xmin), linewidth=3, edgecolor='k',facecolor='none')
            axes[axis_curr_index].add_patch(rect1)
        axes[axis_curr_index].set_title(pred_title, fontsize=30)
        axes[axis_curr_index].axis('off')
        axis_curr_index += 1

    if axis_curr_index < len(axes):
        c = np.max(confidences, axis=2)
        axes[axis_curr_index].imshow(c, confidence_heatmap_viz_cmap, confidence_heatmap_viz_norm, interpolation='nearest')
#         if rect is not None:
#             rect2 = mpl.patches.Rectangle((ymin, xmin), (ymax - ymin), (xmax - xmin), linewidth=3, edgecolor='k',
#                                           facecolor='none')
#             axes[axis_curr_index].add_patch(rect2)
        axes[axis_curr_index].set_title(conf_title, fontsize=30)
        axes[axis_curr_index].axis('off')
        axis_curr_index += 1

    mpl.pyplot.savefig(output_path, pad_inches=0, bbox_inches='tight', dpi=150)
    mpl.pyplot.close('all')

def read_image(image_path):
    image = (np.load(image_path) * 255).astype(np.uint8)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image

def read_saved_frame(pred_dir, image_id):
    states_to_save = ['', 'false_positive', 'false_negative', 'large_object_false_negative', 'true_positive', 'true_negative']
    frame = None
    for state in states_to_save:
        if os.path.isfile(os.path.join(pred_dir, state, image_id+'.png')):
            frame = cv2.imread(os.path.join(pred_dir, state, image_id+'.png'))
            break
        if os.path.isfile(os.path.join(pred_dir, state, image_id+'.jpg')):
            frame = cv2.imread(os.path.join(pred_dir, state, image_id+'.jpg'))
            break
    return frame

def read_images(pred_dir, _id):
    if not os.path.isfile(os.path.join(pred_dir, _id+'_image.npy')):
        return None, None, None, None
    image = np.load(os.path.join(pred_dir, _id+'_image.npy'))
    
    # 100m capped depth
    depth = np.load(os.path.join(pred_dir, _id+'_depth.npy'))
#     # raw depth
#     raw_depth_dir = '/raum_raid/li.yu/data/Jupiter_rock_demo_2021/Jupiter_rock_demo_loamy06_Oct20_2021/model_processed_v4.1_sky_2e-3_lr_1e-3_color_aug_full_model_LR_consistency_regularization_0.2_epoch_23/images/'
#     stereo_data = np.load(os.path.join(raw_depth_dir, _id, 'stereo_output.npz'))
#     depth = stereo_data['point_cloud'][:,:,-1]
#     depth = normalize_and_clip_depth(depth, 200)
    
    pred_label = np.load(os.path.join(pred_dir, _id+'_pred_label.npy'))
    confidence = np.load(os.path.join(pred_dir, _id+'_confidence.npy'))
    return image, depth, pred_label, confidence

# def create_frame(pred_dir, pred_merged_dir, _id, recreate=False):
#     canvas_path = os.path.join(pred_merged_dir, _id+'.png')
#     if recreate:
#         image, depth, pred_label, confidence = read_images(pred_dir, _id)
#         if image is None:
#             return None
#         create_mpl_viz_outputs(canvas_path, image, pred_label, confidence, depth)
#     frame = cv2.imread(canvas_path)
#     return frame

def get_bbox_coords(i=-1, bbox_range_list=[], bbox_coord_list=[]):
    for bi in range(len(bbox_range_list)):
        bbox_range = bbox_range_list[bi]
        if bbox_range[0] <= i <= bbox_range[1]:
            return bbox_coord_list[bi]
    return []

def process_frame(pred_dir, pred_merged_dir, _id, recreate=False, bbox_coords=[]):
    image, depth, pred_label, confidence = None, None, None, None
    l = 0.0
    avg_pixel = 0.0
    bbox_conf = None
    if recreate:
        image, depth, pred_label, confidence = read_images(pred_dir, _id)
        if image is None:
            return image, depth, pred_label, confidence, l, avg_pixel, bbox_conf
        # calculate brightness
        hlsImg = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
        l = np.average(hlsImg[:,:,1])
        image_title = 'Image (brightness: {:.4f})'.format(l)
        # calculate average pixel value at bbox area
        if bbox_coords:
            ymin, ymax, xmin, xmax = bbox_coords
            bbox_pred = pred_label[ymin:ymax+1, xmin:xmax+1]
            bbox_conf = confidence[ymin:ymax+1, xmin:xmax+1]
            avg_pixel = np.count_nonzero(bbox_pred == 1)
    return image, depth, pred_label, confidence, l, avg_pixel, bbox_conf

def create_frame(pred_dir, pred_merged_dir, _id, recreate=False, bbox_coords=[]):
    canvas_path = os.path.join(pred_merged_dir, _id+'.png')
    image, depth, pred_label, confidence = None, None, None, None
    l = 0.0
    avg_pixel = 0.0
    bbox_conf = None
    if recreate:
        image, depth, pred_label, confidence = read_images(pred_dir, _id)
        if image is None:
            return None, None, None, None, None, None, None, None
        # calculate brightness
        hlsImg = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
        l = np.average(hlsImg[:,:,1])
        image_title = 'Image (brightness: {:.4f})'.format(l)
        # calculate average pixel value at bbox area
        if bbox_coords:
            ymin, ymax, xmin, xmax = bbox_coords
            bbox_pred = pred_label[ymin:ymax+1, xmin:xmax+1]
            bbox_conf = confidence[ymin:ymax+1, xmin:xmax+1]
            avg_pixel = np.count_nonzero(bbox_pred == 1)
        create_mpl_viz_outputs(canvas_path, image, pred_label, confidence, depth, image_title=image_title, bbox_coords=bbox_coords)
    frame = cv2.imread(canvas_path)
    return frame, image, depth, pred_label, confidence, l, avg_pixel, bbox_conf

def create_diff_frame(pred_merged_dir, _id, image1, depth1, pred_label1, confidence1, 
                      image2, depth2, pred_label2, confidence2, pred_title='prediction', conf_title='confidence'):
    canvas_path = os.path.join(pred_merged_dir, _id+'_diff.png')
    image = np.abs(image1 - image2)
    depth = np.abs(depth1 - depth2)
    pred_label = np.abs(pred_label1 - pred_label2)
    confidence = np.abs(confidence1 - confidence2)
    create_mpl_viz_outputs(canvas_path, image, pred_label, confidence, depth, conf_title=conf_title)
    frame = cv2.imread(canvas_path)
    return frame

def read_raw_image_by_id(data_dir, _id):
    image_path = os.path.join(data_dir, 'images', _id, 'artifact_debayeredrgb_0_'+_id+'.png')
    image = cv2.imread(image_path)
    return image

def read_raw_image_by_row(data_dir, row):
    image = cv2.imread(os.path.join(data_dir, row.artifact_debayeredrgb_0_save_path))
    return image

def read_rectified_image(data_dir, row):
    data_path = os.path.join(data_dir, row.stereo_pipeline_npz_save_path)
    img = np.load(data_path)['left']
    img = normalize_image(img, hdr_mode=row.hdr_mode, return_8_bit=True)
    return cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

def create_video(ids, pred_dir, video_name, read_func=read_saved_frame, fps=2):
    frame = read_func(pred_dir, ids[10])
    height, width, layers = frame.shape
    print(height, width, layers)

    # .avi MJPG,  .mp4 MP4V
    video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'MP4V'), fps, (width,height), isColor=True)
    
    good = 0
    for _id in tqdm(ids):
        frame = read_func(pred_dir, _id)
        if frame is not None:
            video.write(frame)
            good += 1
    print('total', len(ids), 'used', good)

    # cv2.destroyAllWindows()
    video.release()

## Create video from raw left images or rectified images

In [4]:
# create video from raw left images
data_dir = '/data3/jupiter/datasets/model_positive/halo_human_w_corn_stubble_0812_oct'
# df = pd.read_csv(os.path.join(data_dir, 'annotations.csv'))
df = pd.read_csv(os.path.join(data_dir, 'master_annotations.csv'))

seq_dfs = get_sequences(df, interval=60, per_camera=False, per_camera_pair=True)  # break the data by intervals between sequences
print(df.shape, len(seq_dfs))

/home/li.yu/code/notebooks/utils.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk_df['camera_pair'] = chunk_df['unique_id'].apply(lambda s: s[-7:])
/home/li.yu/code/notebooks/utils.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk_df['camera_pair'] = chunk_df['unique_id'].apply(lambda s: s[-7:])
/home/li.yu/code/notebooks/utils.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

(54273, 156) 132


/home/li.yu/code/notebooks/utils.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk_df['camera_pair'] = chunk_df['unique_id'].apply(lambda s: s[-7:])


### create video

In [10]:
def read_csvs(pred_root_dir, model, labeled_dataset):
    pred_df = pd.read_csv(os.path.join(pred_root_dir, model, labeled_dataset, 'output.csv'))
    if not 'state' in pred_df:
        pred_df['state'] = pred_df['result_state']
    if os.path.isfile(os.path.join(pred_root_dir, model, labeled_dataset, 'dust_ratio.csv')):
        dust_df = pd.read_csv(os.path.join(pred_root_dir, model, labeled_dataset, 'dust_ratio.csv'))
        print(labeled_dataset, pred_df.shape, dust_df.shape)
        df = pred_df[['unique_id', 'state', 'camera_location', 'operation_time']].merge(dust_df, on='unique_id')
    else:
        df = pred_df[['unique_id', 'id', 'state', 'camera_location', 'operation_time']]
    all_cameras = {'Front Pod': ['T01', 'T02', 'T03', 'T04'], 'Right Pod': ['T05', 'T06', 'T07', 'T08'], 'Rear Pod': ['T09', 'T10', 'T11', 'T12'], 'Left Pod': ['T13', 'T14', 'T15', 'T16']}
    all_cameras = {c: pod for pod, cameras in all_cameras.items() for c in cameras}
    df['pod'] = df.camera_location.apply(lambda c: all_cameras[c])
    return df

In [11]:
# load in model predictions
pred_root_dir = '/data/jupiter/li.yu/exps/driveable_terrain_model/'
# model = '9_2_unofficial_rev1_lying_down_sitting_25_ep_prod_weights_10_lo_10_tr_br_dr_p2_v_p1_m_p1_u_dust_0624'
# model = '9_2_unofficial_rev1_lying_down_sitting_25_ep_prod_weights_10_lo_5_tr_br_dr_p2_v_p1_m_p05_u_2_dust_0626'
model = 'kore_5_h'
# unlabeled pred
# pred_df = read_csvs(pred_root_dir, model, os.path.basename(data_dir))
pred_df = read_csvs(pred_root_dir, model, 'model_positive/halo_human_w_corn_stubble_0812_oct')
print(pred_df.shape)
# # labeled pred
# suffix = '_human_labeled_stereo'
# labeled_pred_df = read_csvs(pred_root_dir, model, os.path.basename(data_dir)+suffix)
# print(labeled_pred_df.shape)
# # put labeled rows to raw, unlabeled df
# pred_df.loc[pred_df.unique_id.isin(labeled_pred_df.unique_id), ['total_averaged_dust_conf', 'state']] = labeled_pred_df[['total_averaged_dust_conf', 'state']].values
pred_df.head(2)

(54273, 6)


/tmp/ipykernel_1370113/1295679184.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pod'] = df.camera_location.apply(lambda c: all_cameras[c])


,unique_id,id,state,camera_location,operation_time,pod
0,67256da5a34269d616572b8b_T02_T03,67256da5a34269d616572b8b,true_negative,T02,daytime,Front Pod
1,67256ed799d4d973685d540f_T02_T03,67256ed799d4d973685d540f,false_positive,T02,daytime,Front Pod


In [12]:
seq_df = seq_dfs[6]
pred_seq_df = pred_df[pred_df.id.isin(seq_df.id)]
print(seq_df.shape, pred_seq_df.shape)
pred_seq_df.groupby('state').size()

(505, 158) (505, 6)


state
false_positive      1
true_negative     504
dtype: int64

In [ ]:
# read from multiple cameras and put in one frame
# left_pass_pairs = ['T09_T11', 'T14_T16', 'T14_T15', 'T13_T15']
# right_pass_pairs = ['T05_T07', 'T10_T12', 'T06_T08', 'T06_T07']
# cameras = [f'T{str(i+1).zfill(2)}' for i in range(16)][12:]
# cameras = ['T16', 'T15', 'T14', 'T13', 'T11', 'T09']
# all_cameras = {'front': ['T01', 'T02', 'T03', 'T04'], 'right': ['T05', 'T06', 'T07', 'T08'], 'back': ['T09', 'T10', 'T11', 'T12'], 'left': ['T13', 'T14', 'T15', 'T16']}
all_left_cameras = {'front': ['T01', 'T02'], 'right': ['T05', 'T06'], 'back': ['T09', 'T10'], 'left': ['T13', 'T14']}
# left_cameras = ['T01', 'T02', 'T05', 'T06', 'T09', 'T10', 'T13', 'T14']
# back_left_cameras = ['T09', 'T10']
# left_cameras = {'left_pass': ['T09', 'T14', 'T13', 'T02'], 'right_pass': ['T10', 'T05', 'T06', 'T01']}
# selected = {'front': [1, 3, 5, 11, 14, 15], 'right': [15,], 'back': [], 'left': [6, 10]}
for pod, cameras in all_left_cameras.items():
    # pod = 'all_left'
    # cameras = left_cameras
    print(pod, cameras)
    H = 1  # number of camera rows
    W = 2  # number of camera cols
    for i,seq_df in enumerate(seq_dfs):
        # # select sequence to create video
        # if not i in selected[pod]:
        #     continue
        # # check model prediction and filter by dust/seg outputs
        # pred_seq_df = pred_df[pred_df.id.isin(seq_df.id)]
        # labeled_ids = pred_seq_df.id.to_list()

        cam_dfs = [seq_df[seq_df.camera_location == c] for c in cameras]
        print(i, [len(cdf) for cdf in cam_dfs])
        min_len = min(len(cdf) for cdf in cam_dfs)
        cam_dfs = [cdf.sort_values('collected_on').iloc[:min_len] for cdf in cam_dfs]
        if min_len < 2:
            continue
        # print(i, [cdf.iloc[0].collected_on for cdf in cam_dfs])

        frame = read_raw_image_by_row(data_dir, seq_df.iloc[0])
        height, width, layers = frame.shape
        # print(height, width, layers)

        # .avi MJPG,  .mp4 MP4V
        video_dir = os.path.join(data_dir, f'videos_{len(cameras)}cams')
        os.makedirs(video_dir, exist_ok=True)
        video_name = os.path.join(video_dir, f'{pod}_seq{str(i).zfill(2)}.mp4')
        video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'MP4V'), 3, (width*W,height*H), isColor=True)

        for ii in tqdm(range(min_len)):
            try:
                canvas = np.zeros((height*H, width*W, 3), dtype=np.uint8)
                for ci in range(len(cam_dfs)):
                    cam_df_row = cam_dfs[ci].iloc[ii]
                    frame = read_raw_image_by_row(data_dir, cam_df_row)
                    c = (255,0,0)
                    s = f'{cam_df_row.camera_location} {cam_df_row.collected_on}'
                    # if cam_df_row.id in labeled_ids:
                    #     pred_row = pred_seq_df[pred_seq_df.id == cam_df_row.id].iloc[0]
                    #     s += f' pred: {pred_row.state}, dust: {pred_row.total_averaged_dust_conf:.4f}'
                    #     if pred_row.state == 'true_positive' or pred_row.state == 'false_positive':
                    #         c = (0,0,255)
                    frame = cv2.putText(frame, s, 
                            (40,50), cv2.FONT_HERSHEY_SIMPLEX, 1, c, 2, cv2.LINE_AA)
                    fi, fj = ci // W, ci % W
                    canvas[height*fi:height*(fi+1), width*fj:width*(fj+1)] = frame
                video.write(canvas)
            except:
                print(f'{ii}th image read failed')

        # cv2.destroyAllWindows()
        video.release()
        # break

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


front ['T01', 'T02']
0 [889, 850]


100%|██████████| 850/850 [02:05<00:00,  6.76it/s]
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


1 [365, 361]


100%|██████████| 361/361 [00:53<00:00,  6.70it/s]
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


...

100%|██████████| 365/365 [00:53<00:00,  6.85it/s]
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


2 [1510, 1521]


100%|██████████| 1510/1510 [03:44<00:00,  6.71it/s]
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


3 [487, 477]


 80%|███████▉  | 380/477 [00:56<00:13,  7.01it/s]

In [26]:
# read from rectified data
id = 'unique_id'  # id or unique_id
for i,seq_df in enumerate(seq_dfs):
    # check model prediction and filter by dust/seg outputs
    pred_seq_df = pred_df[pred_df[id].isin(seq_df[id])]
    labeled_ids = pred_seq_df[id].to_list()

    frame = read_rectified_image(data_dir, seq_df.iloc[0])
    height, width, layers = frame.shape
    # print(height, width, layers)

    # .avi MJPG,  .mp4 MP4V
    video_dir = os.path.join(data_dir, f'videos')
    os.makedirs(video_dir, exist_ok=True)
    video_name = os.path.join(video_dir, f'seq{str(i).zfill(3)}_{seq_df.iloc[0].camera_pair}.mp4')
    video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'MP4V'), 6, (width,height), isColor=True)

    for ii in tqdm(range(len(seq_df))):
        try:
            cam_df_row = seq_df.iloc[ii]
            frame = read_rectified_image(data_dir, cam_df_row)
            c = (255,0,0)
            s = f'{cam_df_row.collected_on}'
            if cam_df_row[id] in labeled_ids:
                pred_row = pred_seq_df[pred_seq_df[id] == cam_df_row[id]].iloc[0]
                # s += f' pred: {pred_row.state}, dust: {pred_row.total_averaged_dust_conf:.4f}'
                s += f' pred: {pred_row.state}'
                if pred_row.state == 'true_positive' or pred_row.state == 'false_positive':
                    c = (0,0,255) 
            frame = cv2.putText(frame, s, 
                    (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, c, 1, cv2.LINE_AA)
            video.write(frame)
        except:
            print(f'{ii}th image read failed')

    cv2.destroyAllWindows()
    video.release()
    # break

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
100%|██████████| 516/516 [00:26<00:00, 19.76it/s]
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
100%|██████████| 520/520 [00:19<00:00, 26.03it/s]
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
100%|██████████| 520/520 [00:29<00:00, 17.43it/s]
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
100%|██████████| 505/505 [00:29<00:00, 17.41it/s]
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback